# PROYECTO INTEGRADOR: STEAM
## Tratamiento de los datasets para las funciones

En este notebook se presentan las modificaciones y relaciones entre los datasets obtenidos por el ETL de manera que nos permiten optimizar las funciones requeridas. 


Se intentará en cada caso (y en la medida de lo posible) filtrar las columnas y los registros requeridos para las funciones en este archivo, de manera de que la información llegue lo más limpia posible a la API y no haya inconvenientes por la cantidad de datos.

Para cada función en específico, plantearemos un dataset lo más óptimo posible, que exportaremos desde este archivo pero que importaremos luego en el main.py necesario para fastAPI.

In [56]:
#Importamos librerías
import pandas as pd
import ast

In [57]:
#Importamos el dataset de games
df_games=pd.read_csv('Data/games.csv')
df_games.head()

,genres,title,id,release_year
0,Action,Lost Summoner Kitty,761140,2018
1,Casual,Lost Summoner Kitty,761140,2018
2,Indie,Lost Summoner Kitty,761140,2018
3,Simulation,Lost Summoner Kitty,761140,2018
4,Strategy,Lost Summoner Kitty,761140,2018


In [58]:
#Importamos el dataset de reviews
df_reviews=pd.read_csv('Data/reviews.csv')
df_reviews.head()

,user_id,item_id,recommend,year_review,sentimiento
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,0


In [59]:
#Importamos el dataset de items
df_items=pd.read_csv('Data/items.csv')
df_items.head()

,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,30,Day of Defeat,7
2,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,3830,Psychonauts,333


## **1) Función PlayGenRe**

def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.
#Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

En este caso, vamos a considerar un dataset que solo contengan los géneros, el release_year (año de lanzamiento) y las horas jugadas (playtime_forever). Lo llamaremos: dataset_generos_horas_jugadas.

Como en el dataframe de games tenemos la información correspondiente a los géneros y a los años de lanzamiento, y en el dataframe de items poseemos la información de las horas de juego, vamos a hacer un merge entre ambos dataframe basándonos en el item_id (que el df_games se llama id, y en df_items se llama item_id)

In [60]:
#Se hace un merge entre los dos dataframes
df_generos_horas_jugadas = pd.merge(df_games, df_items, left_on='id', right_on='item_id', how='inner')


In [61]:
#Observamos como nos quedan los datos
df_generos_horas_jugadas.head()

,genres,title,id,release_year,user_id,item_id,item_name,playtime_forever
0,Action,Carmageddon Max Pack,282010,1997,UTNerd24,282010,Carmageddon Max Pack,5
1,Action,Carmageddon Max Pack,282010,1997,saint556,282010,Carmageddon Max Pack,13
2,Action,Carmageddon Max Pack,282010,1997,chidvd,282010,Carmageddon Max Pack,110
3,Action,Carmageddon Max Pack,282010,1997,aerpub,282010,Carmageddon Max Pack,13
4,Action,Carmageddon Max Pack,282010,1997,lucifer666678,282010,Carmageddon Max Pack,8


In [62]:
#Eliminamos las columnas que no vamos a utilizar, y nos quedamos solo con las mencionadas anteriormente.
df_generos_horas_jugadas=df_generos_horas_jugadas.drop(columns=['title','id','item_id','item_name','user_id'])

In [63]:
df_generos_horas_jugadas.head()

,genres,release_year,playtime_forever
0,Action,1997,5
1,Action,1997,13
2,Action,1997,110
3,Action,1997,13
4,Action,1997,8


Como una manera de simplificar la función, modificamos nuestro dataset de manera que funcione de una manera más óptima en el render. Lo que se procede a hacer es, sabiendo que para cada género dado queremos que la función nos sume el playtime_forever por año, por lo que haremos un group by.

In [64]:
#Se agrupa en función de genres y release_year, sumando los playtime_forever y reseteando los índices 
df_generos_horas_jugadas = df_generos_horas_jugadas.groupby(['genres','release_year'])['playtime_forever'].sum().reset_index()

In [65]:
#Verificamos que se hayan agrupado correctamente
df_generos_horas_jugadas

,genres,release_year,playtime_forever
0,Action,1983,3582
1,Action,1984,384
2,Action,1988,16243
3,Action,1989,607
4,Action,1990,18787
...,...,...,...
335,Web Publishing,2013,335849
336,Web Publishing,2014,33732
337,Web Publishing,2015,348861
338,Web Publishing,2016,136


Logramos disminuir nuestro dataset a 3 columnas y 340 filas.

Este dataset lo usaremos para la primera función, por lo que procederemos a exportarlo en csv, para utilizarlo en nuestro archivo main.py

In [66]:
#Exportamos a csv
df_generos_horas_jugadas.to_csv('Data/generos_horas_jugadas.csv', index=False)

## **2) Función UserFonGenre**

def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
#Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}


Para crear el dataframe para esta función haremos un procedimiento parecido al realizado anteriormente. 
En este caso, nos piden que al ingresar un género, la función nos dé el usuario con más horas acumuladas, seguido de una lista de acumulación de horas jugadas por año. 

Los datos son parecidos al caso anterior, necesitamos tomar del df_games los valores de genres y release_year, y del dataframe de items, los valores de playtime_forever y de user_id, por eso procedemos a hacer un merge entre ambos dataframes

In [67]:
#Hacemos un merge entre df_games y df_items
df_usuario_horas_jugadas = pd.merge(df_games, df_items, left_on='id', right_on='item_id', how='inner')

In [68]:
#Revisamos los datos
df_usuario_horas_jugadas.head()

,genres,title,id,release_year,user_id,item_id,item_name,playtime_forever
0,Action,Carmageddon Max Pack,282010,1997,UTNerd24,282010,Carmageddon Max Pack,5
1,Action,Carmageddon Max Pack,282010,1997,saint556,282010,Carmageddon Max Pack,13
2,Action,Carmageddon Max Pack,282010,1997,chidvd,282010,Carmageddon Max Pack,110
3,Action,Carmageddon Max Pack,282010,1997,aerpub,282010,Carmageddon Max Pack,13
4,Action,Carmageddon Max Pack,282010,1997,lucifer666678,282010,Carmageddon Max Pack,8


In [69]:
#Eliminamos las columnas que no vamos a utilizar.
df_usuario_horas_jugadas.drop(columns=['title','id','item_id','item_name'])

,genres,release_year,user_id,playtime_forever
0,Action,1997,UTNerd24,5
1,Action,1997,saint556,13
2,Action,1997,chidvd,110
3,Action,1997,aerpub,13
4,Action,1997,lucifer666678,8
...,...,...,...,...
6887152,Action,2004,76561198208507532,203
6887153,Action,2004,76561198221578852,244
6887154,Action,2004,EnVyIsSmexy,7
6887155,Action,2004,76561198283312749,9


Como en la primer función, en este caso vamos a aprovechar a realizar el agrupamiento para que se sumen las horas acumuladas y así disminuir el tamaño de nuestro dataset. 

In [70]:
#Hacemos group by sumando los playtime
df_usuario_horas_jugadas = df_usuario_horas_jugadas.groupby(['genres', 'release_year','user_id'])['playtime_forever'].sum().reset_index()

In [71]:
#Revisamos el group by
df_usuario_horas_jugadas

,genres,release_year,user_id,playtime_forever
0,Action,1983,2Ta4,18
1,Action,1983,76561197966936422,331
2,Action,1983,76561197968887720,1
3,Action,1983,76561197969020980,98
4,Action,1983,76561197971401137,33
...,...,...,...,...
2844538,Web Publishing,2017,Eosoforcus,58
2844539,Web Publishing,2017,N47H4NI3L,1635
2844540,Web Publishing,2017,dirklah,796
2844541,Web Publishing,2017,kushziller,251


In [72]:
#Exportamos a csv
#df_usuario_horas_jugadas.to_csv('Data/usuario_horas_jugadas.csv', index=False)

Al exportar a csv, el archivo pesa mas que los 50mb recomendados por archivos para github. Por lo tanto decidimos pasarlo a parquet.

In [73]:
df_usuario_horas_jugadas.to_parquet('Data/usuario_horas_jugadas.parquet', engine='pyarrow')

## **3) Función UsersRecommend**

def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

Para este caso, queremos que al ingresar un año en especial, basado en que las recomendaciones sean True, y los review sean positivas o neutrales, nos recomienda los top 3 juegos más recomendados. 

Lo que deberíamos hacer es hacer un merge entre df_reviews de donde tomaremos la columna year_review, recommend y sentimiento (columna creada a partir del analisis de sentimiento) y debemos hacer un merge para obtener el nombre de los titulos de los juegos.

Como una manera de optimizar la consulta, como podría hacer un merge de df_reviews con df_games o con df_items, porque en ambas tengo los títulos, lo haré con df_games que tiene menos datos para optimizar el tiempo de ejecución.

In [74]:
#Hacemos el merge entre df_games y df_reviews en función de la columna id e item_id que tienen la misma información
df_recomendaciones_positivas=pd.merge(df_games, df_reviews, left_on='id', right_on='item_id', how='inner')

In [75]:
#Observamos como nos queda el df
df_recomendaciones_positivas.head()

,genres,title,id,release_year,user_id,item_id,recommend,year_review,sentimiento
0,Action,Half-Life,70,1998,exiaez,70,True,2015,0
1,Action,Half-Life,70,1998,mrpfresh,70,True,2011,0
2,Action,Half-Life,70,1998,armouredmarshmallow,70,True,2014,0
3,Action,Half-Life,70,1998,Bluegills,70,True,2013,2
4,Action,Half-Life,70,1998,76561198071955492,70,True,2013,0


Elijo quedarme con la columna de year_review, en vez de release_year, porque como vamos a considerar las recomendaciones, el año de la recomendación es mucho más acertado para el análisis que deseamos hacer (puede que en el año de salida del juego aún no haya recomendaciones). Procedemos a eliminar release_year y otras columnas que no utilizaremos para esta función.

In [76]:
#Eliminamos las columnas que no utilizaremos para esta función
df_recomendaciones_positivas=df_recomendaciones_positivas.drop(columns=['genres','id', 'release_year','user_id','item_id'])

In [77]:
df_recomendaciones_positivas.head()

,title,recommend,year_review,sentimiento
0,Half-Life,True,2015,0
1,Half-Life,True,2011,0
2,Half-Life,True,2014,0
3,Half-Life,True,2013,2
4,Half-Life,True,2013,0


In [78]:
#Contamos los valores True y False de la columna recommend
df_recomendaciones_positivas['recommend'].value_counts()

recommend
True     92737
False    10509
Name: count, dtype: int64

Para optimizar aún más la consulta, vamos a filtrar nuestro dataframe y vamos a quedarnos en primera instancia con los recommend=True para las recomendaciones positivas.

In [79]:
#Para este caso, me elijo con los recommend que sean True, y los filtro
df_recomendaciones_positivas=df_recomendaciones_positivas[df_recomendaciones_positivas['recommend']==True]

In [80]:
#Vuelvo a hacer el conteo, para verificar que mi df solo tenga los valores True
df_recomendaciones_positivas['recommend'].value_counts()

recommend
True    92737
Name: count, dtype: int64

Ahora también vamos a filtrar los valores donde sentimiento sea 1 o 2 (neutral o positivo) por lo que eliminamos donde la columna sentimiento sea 0.

In [81]:
#Verifico ahora, para los recommend = True, el conteo de los valores de la columna sentimiento
df_recomendaciones_positivas['sentimiento'].value_counts()

sentimiento
2    56527
1    19549
0    16661
Name: count, dtype: int64

In [82]:
#Me quedo solo en mi df con los valores de sentimiento que sean distintos de 0, 1 y 2 (positivo y neutral)
df_recomendaciones_positivas=df_recomendaciones_positivas[df_recomendaciones_positivas['sentimiento']!=0]

In [83]:
#Verifico el conteo de los valores nuevamente para ver el cambio
df_recomendaciones_positivas['sentimiento'].value_counts()

sentimiento
2    56527
1    19549
Name: count, dtype: int64

In [84]:
#Revisamos el df
df_recomendaciones_positivas

,title,recommend,year_review,sentimiento
3,Half-Life,True,2013,2
7,Half-Life,True,2015,1
8,Half-Life,True,2014,2
9,Half-Life,True,2015,2
10,Half-Life,True,2014,2
...,...,...,...,...
103238,Counter-Strike: Condition Zero,True,2011,2
103239,Counter-Strike: Condition Zero,True,2014,2
103242,Counter-Strike: Condition Zero,True,2013,2
103243,Counter-Strike: Condition Zero,True,2015,2


Hemos optimizado de la mejor manera posible el dataframe, ahora lo exportaremos para usar en las funciones

In [85]:
#Exportamos a csv
df_recomendaciones_positivas.to_csv('Data/recomendaciones_positivas.csv', index=False)

 ## **4) Función UsersNotRecommend**

def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

En este caso, es análogo al anterior, pero al hacer el merge entre df_reviews y df_games, me voy a quedar con las recomendaciones que sean falsas, y los sentimiento igual a 0. 

In [86]:
#Hago el merge entre df_games y df_reviews a través de la columna id e item_id que tienen la misma información
df_recomendaciones_negativas=pd.merge(df_games, df_reviews, left_on='id', right_on='item_id', how='inner')

In [87]:
#Vemos el resultado del merge
df_recomendaciones_negativas

,genres,title,id,release_year,user_id,item_id,recommend,year_review,sentimiento
0,Action,Half-Life,70,1998,exiaez,70,True,2015,0
1,Action,Half-Life,70,1998,mrpfresh,70,True,2011,0
2,Action,Half-Life,70,1998,armouredmarshmallow,70,True,2014,0
3,Action,Half-Life,70,1998,Bluegills,70,True,2013,2
4,Action,Half-Life,70,1998,76561198071955492,70,True,2013,0
...,...,...,...,...,...,...,...,...,...
103241,Action,Counter-Strike: Condition Zero,80,2004,76561198023226082,80,False,2014,1
103242,Action,Counter-Strike: Condition Zero,80,2004,76561198030797558,80,True,2013,2
103243,Action,Counter-Strike: Condition Zero,80,2004,Deadly909,80,True,2015,2
103244,Action,Counter-Strike: Condition Zero,80,2004,76561198099442511,80,True,2014,1


Como en el caso anterior, para basarme en un análisis de recomendaciones, considero el year_review y no el year_release por lo que procederé a eliminarlo.

In [88]:
#Eliminamos las columnas que no utilizaremos para esta función
df_recomendaciones_negativas=df_recomendaciones_negativas.drop(columns=['genres','id', 'release_year','user_id','item_id'])

In [89]:
#Cuento los valores que tengo en la columna recommend
df_recomendaciones_negativas['recommend'].value_counts()

recommend
True     92737
False    10509
Name: count, dtype: int64

In [90]:
#El df de recomendaciones negativas, solo mantendrá los valores donde recommend sea falso
df_recomendaciones_negativas=df_recomendaciones_negativas[df_recomendaciones_negativas['recommend']==False]

In [91]:
#Verifico que me haya quedado con los valores falsos
df_recomendaciones_negativas['recommend'].value_counts()

recommend
False    10509
Name: count, dtype: int64

Ahora vamos a optimizar la columna de sentimiento, para un dataframe que solo contiene las recomendaciones que son False y vamos a intentar quedarnos con los valores donde sentimiento = 0.

In [92]:
#Ahora reviso que para recommend=False, los valores de la columna sentimiento
df_recomendaciones_negativas['sentimiento'].value_counts()

sentimiento
0    5355
2    2842
1    2312
Name: count, dtype: int64

In [93]:
#Para este df, solo mantengo los valores de la columna sentimiento que sean 0
df_recomendaciones_negativas=df_recomendaciones_negativas[df_recomendaciones_negativas['sentimiento']==0]

In [94]:
#Verifico que se hayan eliminado los valores
df_recomendaciones_negativas['sentimiento'].value_counts()

sentimiento
0    5355
Name: count, dtype: int64

In [95]:
#Reviso como me queda el dataframe
df_recomendaciones_negativas

,title,recommend,year_review,sentimiento
65,The Ship: Murder Party,False,2014,0
85,The Ship: Murder Party,False,2014,0
105,The Ship: Murder Party,False,2014,0
121,DEFCON,False,2014,0
129,DEFCON,False,2014,0
...,...,...,...,...
103151,Team Fortress Classic,False,2015,0
103157,Team Fortress Classic,False,2015,0
103160,Half-Life: Opposing Force,False,2014,0
103181,Counter-Strike,False,2015,0


In [96]:
#Exportamos a csv
df_recomendaciones_negativas.to_csv('Data/recomendaciones_negativas.csv', index=False)

## **5) Función sentiment_analysis**

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
#Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

Para este caso, necesito que al ingresar el año, me devuelva una lista con la cantidad de registros según sentimiento que haya en cada caso. Por lo ranto, necesito que mi dataframe específico para esta función tenga el nombre del juego, el year_review y el sentimiento.

En mi df_reviews, tomaré el year_review y el sentimiento, pero me faltan los títulos, así que haré un merge, en este caso con df_games, que es un dataset más pequeño y me permite optimizar la operación.

In [97]:
#Hago el merge entre df_games y df_reviews a partir de la columna id e item_id que tienen la misma información
df_sentiment_analysis=pd.merge(df_games, df_reviews, left_on='id', right_on='item_id', how='inner')

In [98]:
#Verifico el dataset obtenido
df_sentiment_analysis.head()

,genres,title,id,release_year,user_id,item_id,recommend,year_review,sentimiento
0,Action,Half-Life,70,1998,exiaez,70,True,2015,0
1,Action,Half-Life,70,1998,mrpfresh,70,True,2011,0
2,Action,Half-Life,70,1998,armouredmarshmallow,70,True,2014,0
3,Action,Half-Life,70,1998,Bluegills,70,True,2013,2
4,Action,Half-Life,70,1998,76561198071955492,70,True,2013,0


Usaremos el año de lanzamiento (release_year) porque el enunciado lo pide. Particularmente creo que tendría más sentido hacerlo con el year_review

In [99]:
#Eliminamos las columnas que no usaremos
df_sentiment_analysis=df_sentiment_analysis.drop(columns=['genres','id', 'year_review','user_id','item_id','recommend','title'])

In [100]:
#Verificamos que las columnas se hayan elimnado correctamente
df_sentiment_analysis

,release_year,sentimiento
0,1998,0
1,1998,0
2,1998,0
3,1998,2
4,1998,0
...,...,...
103241,2004,1
103242,2004,2
103243,2004,2
103244,2004,1


In [101]:
#Exportamos a csv
df_sentiment_analysis.to_csv('Data/sentiment_analysis.csv', index=False)